In [163]:
import re
from math import ceil
with open('input2.txt') as f:
    file = f.read()

bp_dict = {}
for l in file.split('\n'):
    r = re.match('Blueprint ([0-9]+): Each ore robot costs ([0-9]+) ore. Each clay robot costs ([0-9]+) ore. Each obsidian robot costs ([0-9]+) ore and ([0-9]+) clay. Each geode robot costs ([0-9]+) ore and ([0-9]+) obsidian.', l)
    r = [int(x) for x in r.groups()]
    t_d = {}
    t_d['o_r'] = {'o': r[1]}
    t_d['c_r'] = {'o': r[2]}
    t_d['ob_r'] = {'o': r[3], 'c': r[4]}
    t_d['g_r'] = {'o': r[5], 'ob': r[6]}
    bp_dict[r[0]] = t_d

robot_resource_mapping = {
    'o_r': 'o',
    'c_r': 'c',
    'ob_r': 'ob',
    'g_r': 'g'
}

resource_robot_mapping = {v:k for k,v in robot_resource_mapping.items()}

start_robots = {
    'o_r': 1,
    'c_r': 0,
    'ob_r': 0,
    'g_r': 0
}

start_resources = {
    'o': 0,
    'c': 0,
    'ob': 0,
    'g': 0
}

def check_available_resources(robot_schema, resources):
    for r, amt in robot_schema.items():
        if resources[r] < amt:
            return False
    return True

def check_available_robots(robot_schema, robots):
    # robot schema, e.g. {'o': 4}
    for r in robot_schema.keys():
        if robots[f'{r}_r'] <= 0:
            return False
    return True

def consume_resources(robot_schema, resources):
    for r, amt in robot_schema.items():
        resources[r] -= amt

def increment_resources(robots, resources):
    for r, amt in robots.items():
        resources[r.strip('_r')] += amt

def how_long_to_wait(resources, robot_schema, robots, robot_name):
    resources = resources.copy()
    robots = robots.copy()
    time = 0
    # make sure to execute only after check_available_robots is evaluated to be True
    if check_available_resources(robot_schema, resources):
        consume_resources(robot_schema, resources)
        increment_resources(robots, resources)
        robots[robot_name] += 1
        time = 1
    else:
        if check_available_robots(robot_schema, robots):
            add_resource_req = {}
            for res, amt in robot_schema.items():
                add_resource_req[res] = amt - resources[res]
            time = []
            for res, amt in add_resource_req.items():
                time.append(ceil(amt / robots[f'{res}_r']))
            time = max(time) + 1

            increment_resources({k:v*(time ) for k, v in robots.items()}, resources)
            consume_resources(robot_schema, resources)
            robots[robot_name] += 1
        else:
            # increment_resources(robots, resources)
            # time = 1
            return None

    return resources, robots, time



In [164]:
how_long_to_wait({'o': 1, 'c': 0, 'ob': 0, 'g': 0}, {'o': 2}, {'o_r': 1, 'c_r': 1, 'ob_r': 0, 'g_r': 0}, 'ob_r')

({'o': 1, 'c': 2, 'ob': 0, 'g': 0},
 {'o_r': 1, 'c_r': 1, 'ob_r': 1, 'g_r': 0},
 2)

In [165]:
rev_r_lst = list(robot_resource_mapping.keys())[::-1]
rev_r_lst

['g_r', 'ob_r', 'c_r', 'o_r']

In [168]:
def get_key(resources):
    i = 0
    for val in resources.values():
        i = i * 100 + val
    return i

def r(bp, resources, robots, min_, maxtime, state, bp_max):
    max_geode = 0
    if min_ > maxtime:
        return resources['g']

    for robot_name in rev_r_lst:
        if robots[robot_name] >= bp_max[robot_name.strip('_r')]:
            continue
        tmp_tuple = how_long_to_wait(resources, bp[robot_name], robots, robot_name)
        if tmp_tuple is not None:
            upt_res, upt_rob, time = tmp_tuple
            if min_ + time > maxtime:
                continue
            print(resources, robots, min_, robot_name)
            print(upt_res, upt_rob, time, min_ + time , robot_name)
            print()
            max_geode = max(max_geode, r(bp, upt_res, upt_rob, min_ + time, maxtime, state, bp_max))

            # if we can build a geode robot, don't bother building further robots
            if robot_name == 'g_r':
                break

    return max_geode

bp = bp_dict[1]
bp_max = {'o': 0, 'c': 0, 'ob': 0, 'g': 0}
for rob in bp.values():
    for k, v in rob.items():
        bp_max[k] = max(bp_max[k], v)
r(bp, start_resources, start_robots, 0, 5, {}, bp_max)

{'o': 0, 'c': 0, 'ob': 0, 'g': 0} {'o_r': 1, 'c_r': 0, 'ob_r': 0, 'g_r': 0} 0 c_r
{'o': 1, 'c': 0, 'ob': 0, 'g': 0} {'o_r': 1, 'c_r': 1, 'ob_r': 0, 'g_r': 0} 3 3 c_r

{'o': 1, 'c': 0, 'ob': 0, 'g': 0} {'o_r': 1, 'c_r': 1, 'ob_r': 0, 'g_r': 0} 3 c_r
{'o': 1, 'c': 2, 'ob': 0, 'g': 0} {'o_r': 1, 'c_r': 2, 'ob_r': 0, 'g_r': 0} 2 5 c_r

{'o': 0, 'c': 0, 'ob': 0, 'g': 0} {'o_r': 1, 'c_r': 0, 'ob_r': 0, 'g_r': 0} 0 o_r
{'o': 1, 'c': 0, 'ob': 0, 'g': 0} {'o_r': 2, 'c_r': 0, 'ob_r': 0, 'g_r': 0} 5 5 o_r



0

In [82]:
def get_key(resources):
    i = 0
    for val in resources.values():
        i = i * 100 + val
    return i

get_key({
    'o': 24,
    'c': 24,
    'ob': 24,
    'g': 24
})


24242424

In [69]:
import sys
sys.getsizeof([x for x in range(24**4)])

2678088